![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20BartForZeroShotClassification.ipynb)

## Import BartForZeroShotClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 4.4.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import Bart models trained/fine-tuned for sequence classification via `BartForSequenceClassification` or `TFBartForSequenceClassification`. We can use these models for zero-shot classification.
  - These models are usually under `Zero-Shot Classification` category and have `bart` in their labels
  - For zero-shot classification, We will usually use models trained on the nli data sets for best performance.
- Reference: [TFBartForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/bart#transformers.TFBartForSequenceClassification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.38.2 `. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.38.2 tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 35.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow-datasets 4.9.4 requires protobuf>=3.20, but you have

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [facebook/bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli) model from HuggingFace as an example
  - For zero-shot classification, We will usually use models trained on the (m)nli data set for best performance.
- In addition to `TFBartForSequenceClassification` we also need to save the `BartTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [5]:
from transformers import TFBartForSequenceClassification, BartTokenizer
import tensorflow as tf

MODEL_NAME = 'facebook/bart-large-mnli'

tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFBartForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  model = TFBartForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return {"logits":model(input).logits}

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.
Some non-default generation parameters are set in the model config. These should 

Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {MODEL_NAME}

total 1591756
-rw-r--r-- 1 root root       1197 Mar  3 12:06 config.json
drwxr-xr-x 3 root root       4096 Mar  3 12:05 saved_model
-rw-r--r-- 1 root root 1629942064 Mar  3 12:07 tf_model.h5


In [7]:
!ls -l {MODEL_NAME}/saved_model/1

total 36100
drwxr-xr-x 2 root root     4096 Mar  3 12:06 assets
-rw-r--r-- 1 root root       55 Mar  3 12:06 fingerprint.pb
-rw-r--r-- 1 root root   334899 Mar  3 12:06 keras_metadata.pb
-rw-r--r-- 1 root root 36614800 Mar  3 12:06 saved_model.pb
drwxr-xr-x 2 root root     4096 Mar  3 12:06 variables


In [8]:
!ls -l {MODEL_NAME}_tokenizer

total 1432
-rw-r--r-- 1 root root 456318 Mar  3 12:02 merges.txt
-rw-r--r-- 1 root root    957 Mar  3 12:02 special_tokens_map.json
-rw-r--r-- 1 root root   1188 Mar  3 12:02 tokenizer_config.json
-rw-r--r-- 1 root root 999355 Mar  3 12:02 vocab.json


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `vocab.txt` from the tokenizer
- All we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [12]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/merges.txt {asset_path}

In [11]:
vocabs = tokenizer.get_vocab()
vocabs = sorted(vocabs, key=vocabs.get)
with open(f'{asset_path}/vocab.txt', 'w') as f:
  for item in vocabs:
    f.write("%s\n" % item)

In [13]:
# get label2id dictionary
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt`, `merges.txt` and `labels.txt` inside assets directory

In [14]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 852
-rw-r--r-- 1 root root     32 Mar  3 12:11 labels.txt
-rw-r--r-- 1 root root 456318 Mar  3 12:11 merges.txt
-rw-r--r-- 1 root root 407065 Mar  3 12:10 vocab.txt


## Import and Save BartForZeroShotClassification  in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
print("Restart Here")
while True:
  pass

In [1]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.0
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.0
  Preparing metadata (setup.py) ... done


Let's start Spark with Spark NLP included via our simple `start()` function

In [2]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `BartForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BartForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [4]:
from sparknlp.annotator import *
from sparknlp.base import *

MODEL_NAME = 'facebook/bart-large-mnli'

zero_shot_classifier = BartForZeroShotClassification.loadSavedModel(
    '{}/saved_model/1'.format(MODEL_NAME),
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [5]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [6]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your BartForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [7]:
! ls -l {MODEL_NAME}_spark_nlp

total 1626900
-rw-r--r-- 1 root root 1665931316 Mar  3 12:21 bart_classification_tensorflow
drwxr-xr-x 6 root root       4096 Mar  3 12:17 fields
drwxr-xr-x 2 root root       4096 Mar  3 12:17 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForSequenceClassification model 😊

In [9]:
print ("restart Here")
while True:
  pass

restart Here


KeyboardInterrupt: 

In [1]:
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp


spark = sparknlp.start()

MODEL_NAME = 'facebook/bart-large-mnli'
zero_shot_classifier_loaded = BartForZeroShotClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [2]:
from pyspark.ml import Pipeline, PipelineModel

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer().setInputCols("document").setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

text = [["I have a problem with my iphone that needs to be resolved asap!!"],
        ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
        ["I have a phone and I love it!"],
        ["I really want to visit Germany and I am planning to go there next year."],
        ["Let's watch some movies tonight! I am in the mood for a horror movie."],
        ["Have you watched the match yesterday? It was a great game!"],
        ["We need to harry up and get to the airport. We are going to miss our flight!"]]

# create a DataFrame in PySpark
inputDataset = spark.createDataFrame(text, ["text"])
model = pipeline.fit(inputDataset)
model.transform(inputDataset).select("class.result").show()

+---------+
|   result|
+---------+
| [mobile]|
|[weather]|
|[weather]|
| [mobile]|
|[weather]|
| [travel]|
|  [music]|
+---------+



That's it! You can now go wild and use hundreds of
`BartForZeroShotClassification` models as zero-shot classifiers from HuggingFace 🤗 in Spark NLP 🚀